In [1]:
#install gradio
!pip install gradio
!pip install langchain_openai
!pip install gradio_client
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00


Make simple gradio UI, get more advanced below

In [2]:
#imports
import openai
import gradio_client
import gradio as gr
from typing import List, Tuple

#function to be wrapped by gradio UI
def essay_grading(name):
    return "Guten Tag, " + name + "!"

#demo to be shown -- the actual UI part
demo = gr.Interface(
    fn=essay_grading,  #fn is the function being wrapped by the UI
    inputs=["text"],
    outputs=["text"],
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1f5a803faacf2d2321.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


More complex UI part, AI not implemented

In [3]:
#imports
import openai
import gradio_client
import gradio as gr
from typing import List, Tuple

#function to be wrapped by gradio UI
def essay_grading(name):
    return "Guten Tag, " + name + "!"
#test with multiple functions
def essay_grading_negative(name):
    return "NEIN, " + name + "!"


#demo to be shown -- the actual UI part
with gr.Blocks() as demo:  #use gr.Blocks to create more structured layout

  gr.Markdown("# Essay Grader\nver.1.1") # #in markdown means header
  question_textbox = gr.Textbox(label="Input", interactive = True, value = "") #makes textbox at the top of the interface --
  # order in code is order in interface

  with gr.Row(): #puts row-wise layout below what's already made above ^
    #gr.Markdown("# Choose an option; \n slider, textbox, dropdown menu, label, etc. can all be implemented")
    sent_button = gr.Button(value="Send")


  with gr.Column(): #puts column-wise layout below the row (as it's above this ^)
    #gr.Markdown("# Choose an option; \n slider, textbox, dropdown menu, label, etc. can all be implemented\n")
    output_textbox = gr.Textbox(label="Output", interactive=False)
    output_textbox2 = gr.Textbox(label="Output but Bad", interactive=False)

  sent_button.click(
    essay_grading,  #fn is the function being wrapped by the UI
    inputs=[question_textbox],
    outputs=[output_textbox]
   )
  sent_button.click(
    essay_grading_negative,  #fn is the function being wrapped by the UI
    inputs=[question_textbox],
    outputs=[output_textbox2]
   )

demo.launch(debug = True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d9ebaaf9ad3688d9c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1f5a803faacf2d2321.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://d9ebaaf9ad3688d9c3.gradio.live


implement using chatbot -- SIMPLE

In [10]:
#imports
import openai
import gradio_client
import gradio as gr
from typing import List, Tuple

#get OpenAI API key
from google.colab import userdata
openai.api_key = userdata.get('conner').strip()

#function to be wrapped by gradio UI
def essay_grading(prompt: str, question: str, temp = 1.0) -> str:
    essay_prompt = "Give constructive feedback on this essay: " #prompt for the chatbot to follow
    response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
              {"role": "system", "content": essay_prompt}, #dictionaries to define chatbot behavior
              {"role": "user", "content": question} # 3 main roles; system, user, assistant
        ],
            temperature = temp,
            max_tokens=200,
    )
    query_reply = response.choices[0].message.content  #generate response based on first choice given, give the text

    return query_reply #return the chatbot reply




with gr.Blocks() as demo:
  gr.Markdown("# Essay Grader\nver.2.0")

  chatbot = gr.Chatbot() #initialize chatbot
  essay_prompt = "Give constructive feedback on this essay: " #prompt for the chatbot to follow
  prompt_textbox = gr.Textbox(label="Prompt", value=essay_prompt, visible=False) #invisble textbox for the prompt to sit in


  question_textbox = gr.Textbox(label="Input", interactive = True, value = "")
  with gr.Row():
    sent_button = gr.Button(value="Send")
    reset_button = gr.Button(value="Reset")
  with gr.Column():
    output_textbox = gr.Textbox(label="Output", interactive=False)

  sent_button.click(
    essay_grading,
    inputs=[prompt_textbox, question_textbox],
    outputs=[output_textbox]
   )


demo.launch(debug = True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://47fcb41a23dbd99beb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://47fcb41a23dbd99beb.gradio.live


display output through chatbot; remember conversation and reset button working; basically hw2

In [23]:
def essay_grading(prompt: str, question: str, temp = 1.0) -> List[Tuple[str, str]]:  #changed output type to List[Tuple[str, str]] format for
                                                                                     #chatbot preferred output format
    global conversation_history #global variable to hold previous conversations
    conversation_history = [] #initialize as empty list

    conversation_history.append({'role': 'user', 'content': question}) #add previous conversation to chatbot context


    essay_prompt = "Give constructive feedback on this essay: "
    response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "system", "content": prompt}, #reload the prompt to keep the bot aware of it's prompt
            *conversation_history], #changed messages to conversation_history to reload previous conversations in chatbot response
            temperature = temp,
            max_tokens=200,
    )
    query_reply = response.choices[0].message.content
    conversation_history.append({'role': 'assistant', 'content': query_reply})

    conversation_output = [] #empty conversation_output variable to put the output in and display

    for i in range(0, len(conversation_history)-1, 2):   #scrolls thru and seperates user message and assistant replies from conversation history
        user_message = conversation_history[i]['content']
        assistant_reply = conversation_history[i+1]['content']
        conversation_output.append((user_message, assistant_reply))

    return conversation_output, ""



with gr.Blocks() as demo:
  gr.Markdown("# Essay Grader\nver.2.0")

  chatbot = gr.Chatbot()
  essay_prompt = "Give constructive feedback on this essay: "
  prompt_textbox = gr.Textbox(label="Prompt", value=essay_prompt, visible=False)


  question_textbox = gr.Textbox(label="Input", interactive = True, value = "")
  with gr.Row():
    sent_button = gr.Button(value="Send")
    reset_button = gr.Button(value="Reset")

  sent_button.click(
    essay_grading,
    inputs=[prompt_textbox, question_textbox],
    outputs=[chatbot, question_textbox]  #change output to chatbot to display in the chatbot window
   )

  reset_button.click(
    lambda: ([], ""),  # clear conversation history
    inputs=[],  # clear inputs
    outputs=[chatbot, question_textbox]  # reset chatbot
  )

demo.launch(debug = True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://30be27013624984b08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://30be27013624984b08.gradio.live
